# Results visualization

In [269]:
# INPUT 
results_path = '../docs/results/comparison.csv'
trial_name = "door1080-h1-3iter-stream" # insert here the trial name

In [270]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import functools

# Colorblind friendly colorscheme https://davidmathlogic.com/colorblind/#%23648FFF-%23785EF0-%23DC267F-%23FE6100-%23FFB000
colorscheme = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000'] # px.colors.sequential.Viridis[::3] - import plotly.express as px
symbols = ['circle', 'square', 'x', 'diamond', 'triangle-up']

df = pd.read_csv(results_path)
df.head()

,trial name,items,id,type,threads,completion time,service time,bandwidth,speedup,scalability,efficiency
0,house720-h1-3iter-stream,84,1,seq,1,2545024,30297.90,0,1.00,1.00,1.00
1,house720-h1-3iter-stream,84,1,threads,1,2234643,26602.89,0,1.13,1.00,1.13
2,house720-h1-3iter-stream,84,1,threads,2,1216153,14478.01,0,2.09,1.83,1.04
3,house720-h1-3iter-stream,84,1,threads,3,919745,10949.34,0,2.76,2.42,0.92
4,house720-h1-3iter-stream,84,1,threads,4,720007,8571.51,0,3.53,3.10,0.88


In [271]:
assert trial_name in df["trial name"].unique(), "Trial name not found in results.csv"
df = df[df['trial name'] == trial_name]

## Execution times
Execution time in microseconds for each single function obtained with the execution of src/meter.cpp

|       	| READ FRAME 	| DEQUE 	| GREY 	| DETECT MOTION 	|
|:-----:	|:----------:	|:-----:	|:----:	|:-------------:	|
| 1080P 	|    7247    	|   1   	| 3758 	|      864      	|
|  720  	|    2934    	|   1   	| 1564 	|      308      	|
            
<br>

|       	| OPENCV_BLUR 	| BOX_BLUR 	|   H1   	|   H1_7  	|
|:-----:	|:-----------:	|:--------:	|:------:	|:-------:	|
| 1080P 	|     3846    	|   15128  	| 55775 	| 138558 	|
|  720  	|     1888    	|   6281   	| 24559 	|  61234 	|

In [272]:
mean = df.groupby(["type", "threads"]).mean().reset_index().sort_values(by=["type", "threads"])
std = df.groupby(["type", "threads"]).std().reset_index().sort_values(by=["type", "threads"])

means = {name:d for name, d in mean.groupby(["type"])}

def compare(x, y):
    if x == "omp": return 1
    elif y == "omp": return -1
    else: return x < y

std = {name:d for name, d in std.groupby(["type"])}
types = sorted(means.keys(), key=functools.cmp_to_key(compare)) # sort by type, but "omp" in the last position (because only sometimes it is used)
threads = pd.unique(mean["threads"]).tolist()

In [273]:
def plot_metrics(y, title, xtitle, ytitle, log_x=False, log_y=True, width=800, height=400):
    items = means["seq"]["items"]
    fig = go.Figure()

     # Add ideal metric
    if y == "completion time":
        seq_completion_time = means["seq"]["completion time"].iloc[0]
        ideal_completion_time = [seq_completion_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_completion_time, name="ideal", line=dict(dash="dash")))

        # estimated = [estimated_service_time * items / thread for thread in threads]
        # fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "service time":
        seq_service_time = means["seq"]["service time"].iloc[0]
        ideal_service_time = [seq_service_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_service_time, name="ideal", line=dict(dash="dash")))

        # estimated = [estimated_service_time / thread for thread in threads]
        # fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "efficiency":
        ones = pd.Series(np.ones(len(threads)))
        fig.add_trace(go.Scatter(x=threads, y=ones, name="ideal", line=dict(dash="dash")))
    else:
        fig.add_trace(go.Scatter(x=threads, y=threads, name="ideal", line=dict(dash="dash")))

    # Add the actual metric
    for type in types:
        if type == "seq":
            continue
        fig.add_trace(go.Scatter(x=threads, y=means[type][y], name=type, error_y=dict(type="data", array=std[type][y], visible=True), mode="lines"))

    # Use different symbol and color for each trace
    for i, trace in enumerate(fig.data):
        trace.marker.symbol = symbols[i]
        trace.marker.size = 8
        trace.line.color = colorscheme[i]

    # Update layout
    fig.update_layout(
        width=width,
        height=height,
        title=title,
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        xaxis = dict(
            tickmode = 'array',
            tickvals = threads,
        ),
        legend=dict(
            title="Parallel implementation",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=go.layout.Margin(
            l=0, #left margin
            r=0, #right margin
            b=0, #bottom margin
            t=0  #top margin
        ),
    )

    # Log scale
    if log_x:
        fig.update_xaxes(type="log")

    if log_y:
        fig.update_yaxes(type="log")

    fig.show()

## Metrics

In [274]:
plot_metrics(y="speedup", title="Speedup vs. Threads", xtitle="# Threads", ytitle="Speedup")

In [275]:
plot_metrics(y="service time", title="Service time vs Number of threads", xtitle="# Threads", ytitle="Service Time (µs)")

In [276]:
plot_metrics(y="completion time", title="Completion time vs. Number of threads", xtitle="# Threads", ytitle="Completion Time (µs)")

In [277]:
plot_metrics(y="scalability", title="Scalability vs. Number of threads", xtitle="# Threads", ytitle="Scalability")

In [278]:
plot_metrics(y="efficiency", title="Efficiency vs. Threads", xtitle="# Threads", ytitle="Efficiency")